# Curve Fitting (Regression & Interpolation) and Data Visualization

Skeleton code untuk melakukan fungsi curve fitting dan visualisasi data dengan input CSV.

In [ ]:
# Import library yang diperlukan
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score

## 1. Fungsi untuk Membaca Data CSV

In [ ]:
def load_data(file_path):
    """
    Membaca data dari file CSV
    
    Parameters:
    file_path (str): Path ke file CSV
    
    Returns:
    pd.DataFrame: Data yang dibaca dari CSV
    """
    try:
        df = pd.read_csv(file_path)
        return df
    except Exception as e:
        print(f"Error membaca file: {e}")
        return None

# Contoh penggunaan
# df = load_data('data.csv')
# display(df.head())

## 2. Fungsi untuk Curve Fitting - Regresi Linear

In [ ]:
def linear_regression(x, y):
    """
    Melakukan regresi linear
    
    Parameters:
    x (array): Data independen
    y (array): Data dependen
    
    Returns:
    tuple: (model, y_pred, r2)
    """
    x = np.array(x).reshape(-1, 1)
    model = LinearRegression()
    model.fit(x, y)
    
    y_pred = model.predict(x)
    r2 = r2_score(y, y_pred)
    
    print(f"Koefisien: {model.coef_[0]:.4f}")
    print(f"Intercept: {model.intercept_:.4f}")
    print(f"R²: {r2:.4f}")
    
    return model, y_pred, r2

## 3. Fungsi untuk Curve Fitting - Regresi Polinomial

In [ ]:
def polynomial_regression(x, y, degree=2):
    """
    Melakukan regresi polinomial
    
    Parameters:
    x (array): Data independen
    y (array): Data dependen
    degree (int): Derajat polinomial
    
    Returns:
    tuple: (model, poly_features, y_pred, r2)
    """
    x = np.array(x).reshape(-1, 1)
    
    # Transformasi fitur polinomial
    poly_features = PolynomialFeatures(degree=degree)
    x_poly = poly_features.fit_transform(x)
    
    # Fit model
    model = LinearRegression()
    model.fit(x_poly, y)
    
    # Prediksi
    y_pred = model.predict(x_poly)
    r2 = r2_score(y, y_pred)
    
    print(f"Koefisien: {model.coef_}")
    print(f"Intercept: {model.intercept_:.4f}")
    print(f"R²: {r2:.4f}")
    
    return model, poly_features, y_pred, r2

## 4. Fungsi untuk Interpolasi

In [ ]:
def perform_interpolation(x, y, kind='linear', num_points=100):
    """
    Melakukan interpolasi data
    
    Parameters:
    x (array): Data independen
    y (array): Data dependen
    kind (str): Jenis interpolasi ('linear', 'cubic', etc.)
    num_points (int): Jumlah titik untuk interpolasi
    
    Returns:
    tuple: (x_new, y_new, f)
    """
    # Membuat fungsi interpolasi
    f = interpolate.interp1d(x, y, kind=kind, fill_value="extrapolate")
    
    # Membuat titik-titik baru untuk interpolasi
    x_new = np.linspace(min(x), max(x), num_points)
    y_new = f(x_new)
    
    return x_new, y_new, f

## 5. Fungsi untuk Visualisasi Data

In [ ]:
def plot_data_with_fit(x, y, x_fit=None, y_fit=None, title="Data dan Curve Fitting", 
                       xlabel="X", ylabel="Y", scatter_label="Data Asli", line_label="Fitted Curve"):
    """
    Memvisualisasikan data dan kurva hasil fitting
    
    Parameters:
    x (array): Data independen asli
    y (array): Data dependen asli
    x_fit (array): Data independen hasil fitting/interpolasi
    y_fit (array): Data dependen hasil fitting/interpolasi
    title, xlabel, ylabel (str): Label untuk plot
    scatter_label, line_label (str): Label untuk legenda
    """
    plt.figure(figsize=(10, 6))
    
    # Plot data asli
    plt.scatter(x, y, color='blue', alpha=0.6, label=scatter_label)
    
    # Plot hasil fitting jika tersedia
    if x_fit is not None and y_fit is not None:
        plt.plot(x_fit, y_fit, color='red', linewidth=2, label=line_label)
    
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

## 6. Fungsi untuk Menampilkan Statistik Data

In [ ]:
def display_statistics(data):
    """
    Menampilkan statistik deskriptif dari data
    
    Parameters:
    data (pd.DataFrame): Data yang akan dianalisis
    """
    print("\nStatistik Deskriptif:")
    print(data.describe())
    
    # Cek missing values
    missing = data.isnull().sum()
    if missing.sum() > 0:
        print("\nMissing Values:")
        print(missing[missing > 0])

## 7. Demo Penggunaan

In [ ]:
# Load data contoh atau file CSV
# Jika tidak ada file, gunakan data contoh
try:
    df = load_data('data.csv')
except:
    print("Menggunakan data contoh...")
    # Buat data contoh
    np.random.seed(42)
    x = np.linspace(0, 10, 20)
    y = 2*x + 1 + np.random.normal(0, 1, 20)  # y = 2x + 1 + noise
    df = pd.DataFrame({'x': x, 'y': y})
    
# Tampilkan beberapa baris data
print("Data:")
display(df.head())

# Analisis statistik
display_statistics(df)

# Ambil kolom untuk analisis
x = df['x'].values
y = df['y'].values

# Plot data asli
plot_data_with_fit(x, y, title="Data Asli", scatter_label="Data")

# Regresi linear
print("\nRegresi Linear:")
model_linear, y_pred_linear, r2_linear = linear_regression(x, y)
plot_data_with_fit(x, y, x, y_pred_linear, title="Regresi Linear", line_label="Regresi Linear")

# Regresi polinomial
print("\nRegresi Polinomial (degree=2):")
model_poly, poly_features, y_pred_poly, r2_poly = polynomial_regression(x, y, degree=2)
plot_data_with_fit(x, y, x, y_pred_poly, title="Regresi Polinomial", line_label="Regresi Polinomial")

# Interpolasi
print("\nInterpolasi:")
x_interp, y_interp, interp_func = perform_interpolation(x, y, kind='cubic')
plot_data_with_fit(x, y, x_interp, y_interp, title="Interpolasi Cubic", line_label="Interpolasi")

## 8. Fungsi untuk Menganalisis Data CSV Secara Keseluruhan

In [ ]:
def analyze_csv_data(file_path, x_col, y_col, polynomial_degree=2, interpolation_kind='cubic'):
    """
    Fungsi untuk menganalisis data CSV secara lengkap
    
    Parameters:
    file_path (str): Path ke file CSV
    x_col (str): Nama kolom untuk variabel independen
    y_col (str): Nama kolom untuk variabel dependen
    polynomial_degree (int): Derajat polinomial untuk regresi
    interpolation_kind (str): Jenis interpolasi
    """
    # Load data
    df = load_data(file_path)
    if df is None:
        return
    
    # Cek kolom
    if x_col not in df.columns or y_col not in df.columns:
        print(f"Error: Kolom {x_col} atau {y_col} tidak ditemukan.")
        print(f"Kolom yang tersedia: {df.columns.tolist()}")
        return
    
    # Display statistik
    display_statistics(df[[x_col, y_col]])
    
    # Ekstrak data
    x = df[x_col].values
    y = df[y_col].values
    
    # Plot data asli
    plot_data_with_fit(x, y, title=f"Data {x_col} vs {y_col}", 
                       xlabel=x_col, ylabel=y_col, scatter_label="Data")
    
    # Regresi linear
    print("\nRegresi Linear:")
    model_linear, y_pred_linear, r2_linear = linear_regression(x, y)
    plot_data_with_fit(x, y, x, y_pred_linear, title=f"Regresi Linear: {x_col} vs {y_col}", 
                      xlabel=x_col, ylabel=y_col, line_label="Regresi Linear")
    
    # Regresi polinomial
    print(f"\nRegresi Polinomial (degree={polynomial_degree}):")
    model_poly, poly_features, y_pred_poly, r2_poly = polynomial_regression(x, y, degree=polynomial_degree)
    plot_data_with_fit(x, y, x, y_pred_poly, title=f"Regresi Polinomial: {x_col} vs {y_col}", 
                      xlabel=x_col, ylabel=y_col, line_label="Regresi Polinomial")
    
    # Interpolasi
    print(f"\nInterpolasi ({interpolation_kind}):")
    x_interp, y_interp, interp_func = perform_interpolation(x, y, kind=interpolation_kind)
    plot_data_with_fit(x, y, x_interp, y_interp, title=f"Interpolasi {interpolation_kind}: {x_col} vs {y_col}", 
                      xlabel=x_col, ylabel=y_col, line_label="Interpolasi")
    
    # Bandingkan hasil
    print("\nPerbandingan R² dari metode fitting:")
    print(f"Linear Regression: {r2_linear:.4f}")
    print(f"Polynomial Regression (degree={polynomial_degree}): {r2_poly:.4f}")
    
    return {
        'model_linear': model_linear,
        'model_poly': model_poly,
        'interp_func': interp_func,
        'r2_linear': r2_linear,
        'r2_poly': r2_poly
    }

# Contoh penggunaan
# results = analyze_csv_data('data.csv', 'x_column', 'y_column', polynomial_degree=3)